## 1. Title of this project: TODO



## 2. Introduction

Heart disease is the number 1 cause of deaths worldwide, with no 'cure' of any kind. We can, however, try to identify possible factors that lead to higher risk of heart disease. We obtained a set of data collected by Robert Detrano, (M.D., Ph.D.) from the Long Beach and Cleveland Clinic Foundation. This data set contains a total of 76 attributes, of which 14 are used as primary data. We will be using this data to craft a model that attempts to predict whether or not age, sex, total cholestoral levels, maximum heart rate and number of major vessels affected by flourosopy affect the diagnosis of heart disease. 

## 3. Preliminary exploratory data analysis

In [2]:
library(tidyverse)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [3]:
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
download.file(url, destfile = "data/processed.cleveland.data")

In [4]:
heart_df <- read_csv("data/processed.cleveland.data", col_name = FALSE)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
col <- c("age", "sex","cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal","num")

In [6]:
colnames(heart_df) <- col

In [7]:
head(heart_df)

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0


In [8]:
## me trying to figure stuff out here -> good for graph column names later
#age - age in years
#sex - sex (1 = male, 0 = female)
#cp (del?) - chest pain type (1 = typical angina, 2 = atypical angina, 3 = non-anginal pain, 4 = asymptomatic)
#trestbps (del?) - resting blood pressure (in mm Hg on admission to the hospital)
#chol - serum cholestoral in mg/dl
#fbs (del?) - fasting blood sugar > 120 mg/dl (1 = true; 0 = false)
#restecg (del?) - resting electrocardiographic results (0 = normal, 1 = having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria
#thalach - maximum heart rate achieved
#exang (del?) - exercise induced angina (1 = yes; 0 = no)
#oldpeak (del?) - ST depression induced by exercise relative to rest
#slope (del?) - the slope of the peak exercise ST segment (1 = upsloping, 2 = flat, 3 = downsloping)
#ca - number of major vessels (0-3) colored by flourosopy
#thal (del?) - 3 = normal; 6 = fixed defect; 7 = reversable defect
#num - diagnosis of heart disease (angiographic disease status) (0 = < 50% diameter narrowing, >0 (?) = > 50% diameter narrowing)


In [9]:
filheart_df <- select(heart_df, -cp, -trestbps, -fbs, -restecg, -exang, -oldpeak, -slope, -thal)
head(filheart_df)

age,sex,chol,thalach,ca,num
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
63,1,233,150,0.0,0
67,1,286,108,3.0,2
67,1,229,129,2.0,1
37,1,250,187,0.0,0
41,0,204,172,0.0,0
56,1,236,178,0.0,0


In [10]:
split <- initial_split(filheart_df, prop = 0.7, strata = num)
heart_train <- training(split)
heart_test <- testing(split)
head(heart_train)
head(heart_test)

age,sex,chol,thalach,ca,num
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
63,1,233,150,0.0,0
37,1,250,187,0.0,0
41,0,204,172,0.0,0
57,0,354,163,0.0,0
57,1,192,148,0.0,0
56,0,294,153,0.0,0


age,sex,chol,thalach,ca,num
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
67,1,229,129,2.0,1
56,1,236,178,0.0,0
53,1,203,155,0.0,1
56,1,256,142,1.0,2
44,1,263,173,0.0,0
57,1,168,174,0.0,0


In [78]:
heart_zero <- heart_train %>%
        filter(num ==0) %>%
        mutate(sex = as.factor(sex)) %>%
        mutate(ca = as.integer(ca))
heart_nonzero <- heart_train %>%
        filter(num >=1) %>%
        mutate(num = 1) %>%
        mutate(sex = as.factor(sex)) %>%
        mutate(ca = as.integer(ca))
merged_train <- rbind(heart_zero, heart_nonzero)
head(merged_train)

Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”


age,sex,chol,thalach,ca,num
<dbl>,<fct>,<dbl>,<dbl>,<int>,<dbl>
63,1,233,150,0,0
37,1,250,187,0,0
41,0,204,172,0,0
57,0,354,163,0,0
57,1,192,148,0,0
56,0,294,153,0,0


num,counts
<dbl>,<int>
0,114
1,97


num_counts_train <- merged_train %>%
        group_by(num) %>%
        summarize(counts = n())
averaged <- merged_train %>%
        group_by(num) %>%
        summarize(avg_age = mean(age), avg_chol = mean(chol), avg_thal = mean(thalach))
averaged
merged_train_numcounts <- merged_train %>%
        group_by(num) %>%
        summarize(counts = n())
merged_train_numcounts

Details of variables can be found here
https://archive.ics.uci.edu/ml/datasets/Heart+Disease

## 4. Methods

Using classification, we will use knn to predict the instance of heart disease with our chosen factors. The columns (variables) that we will be using are age (age), sex (sex), cholesterol level (chol), maximum heart rate achieved (thalach), number of major vessels colored by flourosopy (ca), and the diagnosis of heart disease (num). For clarity, sex is a binary variable that is coded as male = 1, woman = 0. Cholesterol is measured in mg/dl. Number of major vessels is measured from 0-3. The diagnosis is initially measured from 0-4: 0 indicating no presence and 1-4 indicated presence of heart disease. We mutated this into a binary variable so that 0 indicates no diagnosis and 1 indicates diagnosis. 

To visualize our results, we will be using scatterplots between two variables and use the color function to indicate the instance of heart disease or not. This will help to explain the combinatory power of the two variables in the prediction of heart disease. 


## 5. Expected outcomes and significance

We expect to find that these variables (age, cholesterol level, maximum heart rate achieved, number of major vessels colored by flourosopy, and sex) affect the likelihood of heart disease in patients. Therefore, we expect to see predictive value. 
                                
Findings such as this could have an impact on how doctors communicate health risks to patients. It may also affect how patients change their daily life as certain risks may have increased predictive value of a variable, such as cholesterol level. 

This could lead to future questions about the compounding effect of these variables and all other variables that serve predictive purposes. Knowing how these chosen variables affect the instance of heart disease, questions about other predictive variables such as blood pressure, smoking, or obesity can also be tested individually and together. Interactions between these variables could be a very important question. 
